In [29]:
import csv

# Specify the filenames for the datasets
enrollments_filename = 'enrollments.csv'
engagement_filename = 'daily_engagement.csv'
submissions_filename = 'project_submissions.csv'

In [30]:
# Read the CSV files and store the data in manageable types
def read_csv(filename):
    with open(filename, 'rt') as f:
        reader = csv.DictReader(f)
        return list(reader)
enrollments = read_csv(enrollments_filename)
daily_engagement = read_csv(engagement_filename)
project_submissions = read_csv(submissions_filename)
print(enrollments[0])
print(daily_engagement[0])
print(project_submissions[0])

{'account_key': '448', 'status': 'canceled', 'join_date': '2014-11-10', 'cancel_date': '2015-01-14', 'days_to_cancel': '65', 'is_udacity': 'True', 'is_canceled': 'True'}
{'acct': '0', 'utc_date': '2015-01-09', 'num_courses_visited': '1.0', 'total_minutes_visited': '11.6793745', 'lessons_completed': '0.0', 'projects_completed': '0.0'}
{'creation_date': '2015-01-14', 'completion_date': '2015-01-16', 'assigned_rating': 'UNGRADED', 'account_key': '256', 'lesson_key': '3176718735', 'processing_state': 'EVALUATED'}


## Fixing data type

In [31]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{'account_key': '448',
 'status': 'canceled',
 'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 'days_to_cancel': 65,
 'is_udacity': True,
 'is_canceled': True}

In [32]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{'acct': '0',
 'utc_date': datetime.datetime(2015, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0,
 'projects_completed': 0}

In [33]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

## Problems in the Data

In [34]:
# Rename the "acct" column in the daily_engagement table to "account_key".

for e in daily_engagement:
    # Check to see if the key have been renamed, to prevent Error
    # When running this multiple times
    if 'acct' in e:
        e['account_key'] = e['acct']
        del e['acct']
daily_engagement[0]['account_key']


'0'

## Investigating the data

In [35]:
# Find the total number of rows and the number of unique students (account keys)
# in each table.

#Define a function to gather the account keys into a set  
def get_unique_students(data):
    return (set([e["account_key"] for e in data]))

enrollment_num_rows = len(enrollments)
print(enrollment_num_rows)           
enrollment_num_unique_students = len(get_unique_students(enrollments))
print(enrollment_num_unique_students)  

engagement_num_rows = len(daily_engagement)
print(engagement_num_rows)             
engagement_num_unique_students = len(get_unique_students(daily_engagement))
print(engagement_num_unique_students)

submission_num_rows = len(project_submissions)
print(submission_num_rows)             
submission_num_unique_students = len(get_unique_students(project_submissions))
print(submission_num_unique_students)  

1640
1302
136240
1237
3642
743


## Missing Engagement Records

In [36]:
# Now we have the number of students enrolled is different from the number of students who engaged with the course.
# I will try to find out why this is the case.
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in get_unique_students(daily_engagement):
        print(enrollment)
        break

{'account_key': '1219', 'status': 'canceled', 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}


In [37]:
# The result from the cell above shows that the student canceled their enrollment on the same day they joined.
# What if there are other suprising cases?
suprising_count = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in get_unique_students(daily_engagement) \
        and enrollment['join_date'] != enrollment['cancel_date']:
        suprising_count += 1
        print(enrollment)
print(suprising_count)

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_udacity': True, 'is_canceled': True}
{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), 'days_to_cancel': 99, 'is_udacity': True, 'is_canceled': True}
{'account_key': '1101', 'status': 'current', 'join_date': datetime.datetime(2015, 2, 25, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_udacity': True, 'is_canceled': False}
3


In [38]:
# What was found are the three test accounts that were created by Udacity.
# Indicated by is_udacity = True.
# Here I remove them from the data since I don't plan to include them in any analysis.

is_udacity = set(e['account_key'] for e in enrollments if e['is_udacity'])

In [39]:
# Making new datasets without the Udacity accounts
# In case I want to return to the original datasats

def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in is_udacity:
            non_udacity_data.append(data_point)
    return non_udacity_data

non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement)) 
print(len(non_udacity_submissions))

1622
135656
3634


## Refining the Question

In [ ]:
# The question I wanted to ask:
# How do numbers in the daily engagement table differ for students who pass the
# first project?

# However, there are many problems to this, most notably that this will compare
# data from different lengths of time.

# Therefore, my revision of the question is to only look at engagement from the 
# first week, and exclude students who cancelwithin a week.

# The paid_students variable contains all students who either
# haven't canceled yet or who remained enrolled for more than 7 days. 
# The keys are account keys, and the values are the date the student enrolled.

paid_students = {}
for enrollment in non_udacity_enrollments:
    account_key = enrollment['account_key']
    enrollment_date = enrollment['join_date']
    if enrollment['days_to_cancel'] is None or enrollment['days_to_cancel'] > 7:
        # IMPORTANT:
        # This statement is to check the latest enrollment date for students who
        # Enrolled multiple times
        if (account_key not in paid_students or enrollment_date > paid_students[account_key]):
            paid_students[account_key] = enrollment_date
print(len(paid_students))

995


In [73]:
# Applying this to the datasets
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)
print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))

1293
134549
3618


## Getting Data for the first Week

In [ ]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    # Important to check if the time difference is positive
    # As we may include records from previous enrollments of the same student
    return time_delta.days > 0 and time_delta.days < 7

In [75]:
# This is a list of rows from the engagement table including only rows where
# the student is one of the paid students I just found, and the date is within
# one week of the student's join date.

paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']
        
    if within_one_week(join_date, engagement_record_date):
         paid_engagement_in_first_week.append(engagement_record)
    
len(paid_engagement_in_first_week)

21508

## Exploring the Data of the first week

In [77]:
from collections import defaultdict
# The library allows us to create a dictionary that returns a default value
# if the key is not found.

# This will have a default value of an empty list. Because we want to 
# Get the list of records for each student.
engagement_by_account = defaultdict(list)
for record in paid_engagement_in_first_week:
    account_key = record['account_key']
    # This is where defultdict comes in handy.
    engagement_by_account[account_key].append(record)

# A simple dictionary to hold the total minutes for each account
total_minutes_by_account = {}
for account_key, records in engagement_by_account.items():
    total_minutes = 0
    for record in records:
        total_minutes += record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [82]:
#Getting just the values for total minutes, account keys are not needed now
total_minutes = list(total_minutes_by_account.values())

import numpy as np
# The libraray to do caclulations on the data 

# Getting the statistics I need from total minutes spent
print('Mean:', np.mean(total_minutes))
print('Standard Deviation:', np.std(total_minutes))
print('Minimum:', np.min(total_minutes))
print('Maximum:', np.max(total_minutes))

Mean: 647.5901738262695
Standard Deviation: 1129.2712104188108
Minimum: 0.0
Maximum: 10568.100867332541
